In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import os
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, TimeDistributed, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from datetime import datetime
from tqdm import tqdm

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
DATA_PATH = '/home/smayan/Desktop/Cricket Pose Estimation /Data'
sequence_length = 30
min_sequences_per_class = 10

In [7]:
actions = np.array(sorted([folder for folder in os.listdir(DATA_PATH) 
                          if os.path.isdir(os.path.join(DATA_PATH, folder))]))
print(f"Detected cricket shots: {actions}")
for action in actions:
    video_files = [f for f in os.listdir(os.path.join(DATA_PATH, action)) if f.endswith(('.mp4', '.avi', '.mov'))]
    print(f"{action}: {len(video_files)} videos")

Detected cricket shots: ['Backfoot punch' 'Cover drive' 'Cut Shot' 'FBD' 'Flick'
 'Front Food defence' 'On Drive' 'Pull Shot' 'Reverse Sweep' 'Stance'
 'Straight Drive' 'Sweep' 'Uppercut' 'loft']
Backfoot punch: 19 videos
Cover drive: 29 videos
Cut Shot: 43 videos
FBD: 15 videos
Flick: 22 videos
Front Food defence: 32 videos
On Drive: 40 videos
Pull Shot: 40 videos
Reverse Sweep: 30 videos
Stance: 42 videos
Straight Drive: 25 videos
Sweep: 27 videos
Uppercut: 29 videos
loft: 31 videos


In [12]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        video_files = [f for f in os.listdir(action_path) if f.endswith(('.mp4', '.avi', '.mov'))]
        action_sequences = []

        for video_file in tqdm(video_files, desc=f"Processing {action}"):
            video_path = os.path.join(action_path, video_file)
            cap = cv.VideoCapture(video_path)
            total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
            stride = max(1, sequence_length // 4)

            for start_frame in tqdm(range(0, total_frames - sequence_length + 1, stride), 
                                    desc=f"Frames in {video_file}", leave=False):
                cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
                sequence = []

                for _ in range(sequence_length):
                    ret, frame = cap.read()
                    if not ret:
                        break
                    frame = cv.resize(frame, (640, 480))
                    image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                    image.flags.writeable = False
                    results = holistic.process(image)
                    image.flags.writeable = True
                    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
                    if results.pose_landmarks:
                        pose = np.array([[res.x, res.y, res.z, res.visibility] 
                                         for res in results.pose_landmarks.landmark]).flatten()
                    else:
                        pose = np.zeros(33*4)
                    sequence.append(pose)

                if len(sequence) == sequence_length:
                    action_sequences.append(sequence)

            cap.release()

        while len(action_sequences) < min_sequences_per_class:
            if action_sequences:
                original_seq = np.array(action_sequences[len(action_sequences) % len(action_sequences)])
                noise = np.random.normal(0, 0.01, original_seq.shape)
                augmented_seq = original_seq + noise
                action_sequences.append(augmented_seq.tolist())

        for seq in action_sequences:
            sequences.append(seq)
            labels.append(label_map[action])
        print(f"Generated {len(action_sequences)} sequences for {action}")

I0000 00:00:1753634054.267567    4932 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1753634054.337772    7482 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 570.172.08), renderer: NVIDIA GeForce RTX 4070 SUPER/PCIe/SSE2
Processing Backfoot punch:   0%|          | 0/19 [00:00<?, ?it/s]W0000 00:00:1753634054.377479    7459 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753634054.401762    7471 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753634054.402800    7467 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753634054.403202    7475 inference_feedback_manager.cc:114] Feedback manager requires a model with a single s

Generated 112 sequences for Backfoot punch


Processing Cover drive: 100%|██████████| 29/29 [01:49<00:00,  3.77s/it]


Generated 152 sequences for Cover drive


Processing Cut Shot: 100%|██████████| 43/43 [02:18<00:00,  3.21s/it]


...

Generated 159 sequences for Sweep


Processing Uppercut: 100%|██████████| 29/29 [01:44<00:00,  3.61s/it]


Generated 146 sequences for Uppercut


Processing loft: 100%|██████████| 31/31 [02:11<00:00,  4.26s/it]

Generated 184 sequences for loft


In [13]:
X = np.array(sequences)
y = np.array(labels)
print(f"Dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")

Dataset shape: (2284, 30, 132)
Labels shape: (2284,)


In [14]:
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
y_categorical = to_categorical(y, num_classes=len(actions))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)


In [16]:
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
model = Sequential()

# TimeDistributed CNN - 1 block only
model.add(TimeDistributed(Conv1D(32, kernel_size=3, activation='relu'), input_shape=(sequence_length, X.shape[2], 1)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Dropout(0.2)))
model.add(TimeDistributed(Flatten()))
model.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)))
model.add(LSTM(32, return_sequences=False, dropout=0.3, recurrent_dropout=0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(len(actions), activation='softmax'))

In [19]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = f'logs/cricket_model_{timestamp}'

In [20]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [21]:
callbacks = [
    TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq='epoch'),
    EarlyStopping(patience=15, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=10, min_lr=1e-7)
]

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/100


I0000 00:00:1753635786.227966   20763 cuda_dnn.cc:529] Loaded cuDNN version 91100


115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.0937 - loss: 2.6333 - val_accuracy: 0.1969 - val_loss: 2.2696 - learning_rate: 0.0010
Epoch 2/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.1625 - loss: 2.3372 - val_accuracy: 0.3282 - val_loss: 1.9912 - learning_rate: 0.0010
Epoch 3/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.2492 - loss: 2.0888 - val_accuracy: 0.3961 - val_loss: 1.7002 - learning_rate: 0.0010
Epoch 4/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.3358 - loss: 1.8538 - val_accuracy: 0.4639 - val_loss: 1.5498 - learning_rate: 0.0010
Epoch 5/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.3994 - loss: 1.6935 - val_accuracy: 0.5164 - val_loss: 1.3462 - learning_rate: 0.0010
Epoch 6/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.4793 - loss: 1.5020 - val_accuracy: 0.5339 - val_loss: 1.2812 - learning_rate: 0.0010
Epoch 7/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.4835 - loss: 1.4736

In [23]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9562


In [24]:
model.save('cricket_pose_mode_simple.h5')
model.save('cricket_pose_model_simple.keras')
np.save('cricket_label_map.npy', label_map)

In [25]:
# Load trained model and label map
model = tf.keras.models.load_model('cricket_pose_model.h5')
label_map = np.load('cricket_label_map.npy', allow_pickle=True).item()
actions = list(label_map.keys())

# Variables for prediction
sequence = []
sequence_length = 30
threshold = 0.7

# Start webcam
cap = cv.VideoCapture('/home/smayan/Desktop/Cricket Pose Estimation /Model Training/WhatsApp Video 2025-07-27 at 16.12.07.mp4')

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize for consistent input
        frame = cv.resize(frame, (640, 480))

        # Detection
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
            )

        # Extract keypoints
        if results.pose_landmarks:
            keypoints = np.array([[res.x, res.y, res.z, res.visibility]
                                  for res in results.pose_landmarks.landmark]).flatten()
        else:
            keypoints = np.zeros(33*4)

        # Append to sequence
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]

        if len(sequence) == sequence_length:
            input_seq = np.expand_dims(np.array(sequence), axis=0)
            input_seq = input_seq.reshape(1, sequence_length, -1, 1)

            # Predict
            res = model.predict(input_seq, verbose=0)[0]
            predicted_action = actions[np.argmax(res)]
            confidence = np.max(res)

            # Show prediction
            if confidence > threshold:
                cv.putText(image, f'{predicted_action}: {confidence:.2f}',
                           (10, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Show probabilities
            for i, (action, prob) in enumerate(zip(actions, res)):
                y_pos = 100 + i * 30
                cv.rectangle(image, (10, y_pos), (int(prob * 300) + 10, y_pos + 25), (0, 255, 0), -1)
                cv.putText(image, f'{action}: {prob:.2f}', (15, y_pos + 18),
                           cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        # Show output
        cv.imshow('Cricket Pose Estimation', image)

        # Quit
        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()


I0000 00:00:1753636258.364997    4932 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1753636258.396638   34415 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 570.172.08), renderer: NVIDIA GeForce RTX 4070 SUPER/PCIe/SSE2
W0000 00:00:1753636258.425987   34394 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753636258.443579   34387 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753636258.444953   34413 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753636258.445092   34387 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000